### Tool Create

In [22]:
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="openai/gpt-oss-120b", temperature=0.3)
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002E638636590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002E638B843A0>, model_name='openai/gpt-oss-120b', temperature=0.3, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [23]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
import requests
from  langchain_core.tools import InjectedToolArg
from typing import Annotated

In [24]:
## tool create 
@tool
def convert_currency(base_currency: str, target_currency: str, amount: float) -> dict:
    """
    Convert an amount from one currency to another.
    First fetches the conversion rate, then calculates the converted amount.
    """
    # Get conversion rate
    url = f'https://v6.exchangerate-api.com/v6/f6e1db3585e2c9faa451de1e/pair/{base_currency}/{target_currency}'
    response = requests.get(url)
    data = response.json()
    
    if data.get('result') == 'success':
        conversion_rate = data.get('conversion_rate')
        converted_amount = amount * conversion_rate
        return {
            'conversion_rate': conversion_rate,
            'converted_amount': converted_amount,
            'base_currency': base_currency,
            'target_currency': target_currency,
            'original_amount': amount
        }
    else:
        return {'error': 'Failed to get conversion rate', 'details': data}

In [26]:
convert_currency.invoke({'base_currency': 'USD', 'target_currency': 'NPR', 'amount': 10})

{'conversion_rate': 145.6441,
 'converted_amount': 1456.441,
 'base_currency': 'USD',
 'target_currency': 'NPR',
 'original_amount': 10.0}

In [28]:
# convert_currency.invoke({'base_currency_value':10, 'conversion_rate': 145.2991})

In [29]:
convert_currency.args

{'base_currency': {'title': 'Base Currency', 'type': 'string'},
 'target_currency': {'title': 'Target Currency', 'type': 'string'},
 'amount': {'title': 'Amount', 'type': 'number'}}

### Tool binding

In [30]:
from langchain_core.messages import HumanMessage, ToolMessage

# Add a system message to guide the LLM on calling both tools in one go
system_prompt = SystemMessage(content="""
You are a currency conversion assistant. When the user asks for a conversion factor and to convert a value:
1. Call 'get_conversion_factor' with base_currency and target_currency to fetch the rate.
2. Call 'convert' with only base_currency_value (the conversion_rate will be injected automatically from the prior call).
You can and should call both tools in a single response for efficiency, as the system handles the injection of the rate into 'convert'.
Do not call only one tool—always call both when both are needed.
""")

# User message
user_message = HumanMessage('What is the conversion factor between USD and NPR, and based on that can you convert 10 USD to NPR')


In [31]:
llm_with_tools = llm.bind_tools([convert_currency])

In [32]:
messages = [system_prompt, user_message]
messages

[SystemMessage(content="\nYou are a currency conversion assistant. When the user asks for a conversion factor and to convert a value:\n1. Call 'get_conversion_factor' with base_currency and target_currency to fetch the rate.\n2. Call 'convert' with only base_currency_value (the conversion_rate will be injected automatically from the prior call).\nYou can and should call both tools in a single response for efficiency, as the system handles the injection of the rate into 'convert'.\nDo not call only one tool—always call both when both are needed.\n", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the conversion factor between USD and NPR, and based on that can you convert 10 USD to NPR', additional_kwargs={}, response_metadata={})]

In [34]:
ai_message = llm_with_tools.invoke(messages)
print(ai_message)
print(ai_message.tool_calls)

content='' additional_kwargs={'reasoning_content': 'We need to call get_conversion_factor and convert. However only tool provided is convert_currency which does both steps. According to developer: we should call get_conversion_factor then convert. But only tool is convert_currency which "First fetches the conversion rate, then calculates". So maybe we just call convert_currency with amount 10, base USD, target NPR. That will return conversion factor and converted amount? It says returns any. Probably returns object with conversion_rate and converted_amount. We\'ll call it.', 'tool_calls': [{'id': 'fc_88d29b88-1802-4f6f-8e36-b15b8e79e4e1', 'function': {'arguments': '{"amount":10,"base_currency":"USD","target_currency":"NPR"}', 'name': 'convert_currency'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 276, 'total_tokens': 420, 'completion_time': 0.354592019, 'prompt_time': 0.013430302, 'queue_time': 0.056425428, 'total_time': 0.368022

In [35]:
messages.append(ai_message)

In [36]:
ai_message.tool_calls

[{'name': 'convert_currency',
  'args': {'amount': 10, 'base_currency': 'USD', 'target_currency': 'NPR'},
  'id': 'fc_88d29b88-1802-4f6f-8e36-b15b8e79e4e1',
  'type': 'tool_call'}]

In [37]:
for tool_call in ai_message.tool_calls:
    # print(tool_call)
    
    ## execute the 1st tool and get the value of conversion rate
    if tool_call['name'] == 'convert_currency':
        tool_messages1 = convert_currency.invoke(tool_call)
        print(tool_messages1)
        # print(tool_messages1.content)
        ##fetch the conversion rate
        conversion_rate = tool_messages1.content
        print(conversion_rate)
        
        messages.append(tool_messages1)
        print(messages)
        
        
    # ## execute the 2nd tool using the conversion rate from tool 1
    # if tool_call['name'] == 'convert':
    #     ## fetch the current args
    #     tool_call['args']['conversion_rate'] = conversion_rate
    #     tool_messages2 = convert.invoke(tool_call)
    #     messages.append(tool_messages2)
    #     print(messages)
        

content='{"conversion_rate": 145.6441, "converted_amount": 1456.441, "base_currency": "USD", "target_currency": "NPR", "original_amount": 10.0}' name='convert_currency' tool_call_id='fc_88d29b88-1802-4f6f-8e36-b15b8e79e4e1'
{"conversion_rate": 145.6441, "converted_amount": 1456.441, "base_currency": "USD", "target_currency": "NPR", "original_amount": 10.0}
[SystemMessage(content="\nYou are a currency conversion assistant. When the user asks for a conversion factor and to convert a value:\n1. Call 'get_conversion_factor' with base_currency and target_currency to fetch the rate.\n2. Call 'convert' with only base_currency_value (the conversion_rate will be injected automatically from the prior call).\nYou can and should call both tools in a single response for efficiency, as the system handles the injection of the rate into 'convert'.\nDo not call only one tool—always call both when both are needed.\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the convers

In [38]:
llm_with_tools.invoke(messages).content

'The current conversion factor from **USD →\u202fNPR** is **1\u202fUSD = 145.6441\u202fNPR**.\n\nConverting **10\u202fUSD** using this rate:\n\n**10\u202fUSD × 145.6441 = 1,456.441\u202fNPR**\n\nSo, **10\u202fUSD ≈ 1,456.44\u202fNPR**.'